# NuGetConsume

NGC is an API to depend on NuGet packages in Dyalog APL.

It takes care of installing from NuGet and some of the managing of `⎕USING` statements.

## Features
- add packages
- get path to .dll for `⎕USING` (additional namespaces might not be possible automatically)
- `NGC.Export'install'` will export .csproj and dependencies will be managed via `dotnet` CLI on user's system
- `NGC.Export'publish'` will export DLLs
- retarget existing project? TBD - might only be able to take packages list and attempt create new .NET project

## Todo
- check can `⎕NEW` on existing .csproj
- document info on platform compatibility
- can we get DLL name from package info? esp. if different from package name...
- can we get assembly namespaces from package info (I think Bjørn said not necessarily)
- tmp path for .csproj by default?
- can we have it *not* be a class?
    - `⎕NEW thing` to carry around info seems nice
    - but end up not hiding anything since all my methods are public?
    - but what other way to hold the info? pass a namespace around to functions? 

## .NET projects
NGC uses the `dotnet` Command Line Interface (CLI) to set up a .NET project which manages obtaining compatible versions of NuGet packages and their dependencies.

The project is defined in a **.csproj** file, while dependencies and paths to assembly DLLs are listed in **obj/project.assets.json**.

## .NET Versions
Dyalog can be configured to use various .NET versions [*Dyalog Forums post*](https://forums.dyalog.com/viewtopic.php?f=22&t=1863)

**Officially supported .NET versions:**
|Dyalog version|.NET version|.NET Framework (Windows only)
|---|---|---|
|17.1|3.1|4,8|
|18.2|3.1|4.8|
|19.0|6.0|4.8|

NGC will create and export .NET packages under a single .NET project targeting the currently running .NET version.

NuGet Gallery package pages ([example: Clock](https://www.nuget.org/packages/Clock#supportedframeworks-body-tab)) list compatible versions under "frameworks" tab. **netstandard** is supported across .NET versions, while **netN** for some number **N** corresponds to a particular .NET version.

## Usage
Create an instance of the NGC class.

In [48]:
]create NGC "C:\g\NuGetConsume\APLSource"

ERRORS ENCOUNTERED: ⎕SE.Link.Create: Cannot link a non-empty namespace to a non-empty directory: #.NGC ←→ C:\g\NuGetConsume\APLSource

In [69]:
]link.refresh #.NGC

Imported: #.NGC ← C:\g\NuGetConsume\APLSource

In [70]:
ngc←⎕NEW NGC.NGC'/tmp/myapp'

***Note:*** Jupyter kernel can't print asynchronous output... [issue 82](https://github.com/Dyalog/dyalog-jupyter-kernel/issues/82)

Or maybe unrelated... why does AddPackages output work?

In [71]:
⎕←ngc.project_dir
⎕←ngc.dotnet_version

/tmp/myapp

net48

In [72]:
ngc.AddPackages 'Clock'

Determining projects to restore...
 Writing C:\Users\rpark\AppData\Local\Temp\tmp4A4.tmp
info : Adding PackageReference for package 'Clock' into project 'C:\tmp\myapp\myapp.csproj'.
info : CACHE https://api.nuget.org/v3/registration5-gz-semver2/clock/index.json
info : Restoring packages for C:\tmp\myapp\myapp.csproj...
warn : NU1701: Package 'Html2Xhtml 1.1.2.4' was restored using '.NETFramework,Version=v4.6.1, .NETFramework,Version=v4.6.2, .NETFramework,Version=v4.7, .NETFramework,Version=v4.7.1, .NETFramework,Version=v4.7.2, .NETFramework,Version=v4.8' instead of the project target framework 'net6.0'. This package may not be fully compatible with your project.
info : Package 'Clock' is compatible with all the specified frameworks in project 'C:\tmp\myapp\myapp.csproj'.
info : PackageReference for package 'Clock' version '1.0.3' updated in file 'C:\tmp\myapp\myapp.csproj'.
info : Assets file has not changed. Skipping assets file writing. Path: C:\tmp\myapp\obj\project.assets.json
log : Restored C:\tmp\myapp\myapp.csproj (in 139 ms).

In [79]:
⎕USING←''
⎕USING,←⎕←ngc.GetUsing'Clock'
Clock
Clock.UtcNow

┌─────────────────────────────────────────────────────────────────────────────┐
│Clock,C:\Users\rpark/.nuget/packages/Clock/1.0.3/lib/netstandard1.3/Clock.dll│
└─────────────────────────────────────────────────────────────────────────────┘

(Clock)

10/07/2023 19:44:20 +00:00